# Membership Inference over Diffusion-models-based Synthetic Tabular Data (MIDST) Challenge @ SaTML 2025.

## White Box Multi Table Competition
Welcome to the MIDST challenge!

The MIDST challenge is a series of competitions aiming to quantitatively evaluate the privacy of synthetic tabular data generated by diffusion models, with a specific focus on its resistance to membership inference attacks (MIAs).

This competition focuses on White Box MIA on tabular diffusion models trained on a multi table transaction dataset. In particular, MIA will be explored over a state-of the art method Multi-relational Tabular Diffusion Model [ClavaDDPM](https://arxiv.org/abs/2405.17724). A collection of ClavaDDPM models will be trained on random subsets of the transaction dataset. The goal is to create an approach (MIA) that can distinguish between samples used to train a model (train data) and other data randomly sampled from the transaction dataset (holdout data) given the model and it's output synthetic data. The `eval` set includes 10 models, each with its own set of challenge points (ie train and holdout data), to evaluate solutions on. To facilitate designing an attack, 30 `train` models are provided with comprehensive information about the model, training data and output synthetic data. Additionally, 10 `dev` models are provided to assist in evaluating the effectiveness of attacks prior to making a final submission to the `eval` set. 

This notebook will walk you through the process of creating and packaging a submission to the white box mutli table challenge.

## Package Imports and Evironment Setup

To start, lets import the required packages and define global variables:

In [7]:
import csv
import os
import random
import zipfile

from pathlib import Path
from functools import partial
from typing import Callable, Any

import numpy as np
import torch

from tqdm.notebook import tqdm
from midst.data import get_challenge_points
from midst.metrics import get_tpr_at_fpr

In [8]:
CLAVADDPM_DATA_DIR = "clavaddpm_white_box"

## Data

Next, lets download and extract the data for the competition:

In [1]:
!gdown 1WzgDhNFySjX3RMTxmDVX5BnXnq9q_sGU
!unzip -qq -o clavaddpm_white_box.zip

Downloading...
From (original): https://drive.google.com/uc?id=1G2gtdwJwbskctVmCgBr98iLG0dVTY4eU
From (redirected): https://drive.google.com/uc?id=1G2gtdwJwbskctVmCgBr98iLG0dVTY4eU&confirm=t&uuid=585965d2-96c3-4e17-a37d-59ebf25d8965
To: /Users/johnjewell/Desktop/github/MIDST/starter_kits/clavaddpm_white_box.zip
100%|███████████████████████████████████████| 16.3G/16.3G [02:34<00:00, 105MB/s]


**Note:** If there is an issue with the download (ie throttled for downloading too many files with gdown) you can simply download the zip manually from this [link](https://drive.google.com/file/d/1WzgDhNFySjX3RMTxmDVX5BnXnq9q_sGU/view?usp=drive_link) and extract it in the same directory this notebook exists.

### Contents
The archives extracted under the `clavaddpm_white_box` contain 3 subdirectories:

- `train`: Comprehensive information (ie model weights+architecture, training data, output synthetic data etc.) about the set of shadow models. Use these to develop your attacks without having to train your own models.
- `dev`: Set of challenge points. Membership predictions for these challenges will be used to evaluate submissions during the competition and update the live scoreboard in CodaBench.
- `final`: Set of challenge points. Membership predictions for these challenges will be used to evaluate submissions when the competition closes and to determine the final ranking.

The contents of the `train`, `dev` and `eval` subdirectory of `clavaddpm_white_box` contain the following files: 

<table> <tr> <th>Model Eval</th> <th>File Name</th> <th>Description</th> </tr> <!-- White-Box Models - Train --> <tr> <td rowspan="36"><strong>White-Box Models - Train</strong></td> <!-- Train data with IDs --> <td>account.csv</td> <td></td> </tr> <tr> <td>card.csv</td> <td></td> </tr> <tr> <td>client.csv</td> <td></td> </tr> <tr> <td>disp.csv</td> <td></td> </tr> <tr> <td>district.csv</td> <td></td> </tr> <tr> <td>loan.csv</td> <td></td> </tr> <tr> <td>order.csv</td> <td></td> </tr> <tr> <td>trans.csv</td> <td></td> </tr> <!-- Data domain files --> <tr> <td>account_domain.json</td> <td></td> </tr> <tr> <td>card_domain.json</td> <td></td> </tr> <tr> <td>client_domain.json</td> <td></td> </tr> <tr> <td>disp_domain.json</td> <td></td> </tr> <tr> <td>district_domain.json</td> <td></td> </tr> <tr> <td>loan_domain.json</td> <td></td> </tr> <tr> <td>order_domain.json</td> <td></td> </tr> <tr> <td>trans_domain.json</td> <td></td> </tr> <!-- Challenge data and labels --> <tr> <td>challenge_with_id.csv</td> <td></td> </tr> <tr> <td>challenge_label.csv</td> <td></td> </tr> <!-- Label encoders --> <tr> <td>account_label_encoders.pkl</td> <td></td> </tr> <tr> <td>card_label_encoders.pkl</td> <td></td> </tr> <tr> <td>client_label_encoders.pkl</td> <td></td> </tr> <tr> <td>disp_label_encoders.pkl</td> <td></td> </tr> <tr> <td>district_label_encoders.pkl</td> <td></td> </tr> <tr> <td>loan_label_encoders.pkl</td> <td></td> </tr> <tr> <td>order_label_encoders.pkl</td> <td></td> </tr> <tr> <td>trans_label_encoders.pkl</td> <td></td> </tr> <tr> <td>workspace/train_1/cluster_ckpt.pkl</td> <td></td> </tr> <tr> <td>workspace/train_1/models/*</td> <td></td> </tr> <!-- Synthetic data --> <tr> <td>workspace/train_1/account/_final/account_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/card/_final/card_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/client/_final/client_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/disp/_final/disp_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/district/_final/district_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/loan/_final/loan_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/order/_final/order_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/trans/_final/trans_synthetic.csv</td> <td></td> </tr> <!-- White-Box Models - Dev --> <tr> <td rowspan="27"><strong>White-Box Models - Dev</strong></td> <!-- Data domain files --> <td>account_domain.json</td> <td></td> </tr> <tr> <td>card_domain.json</td> <td></td> </tr> <tr> <td>client_domain.json</td> <td></td> </tr> <tr> <td>disp_domain.json</td> <td></td> </tr> <tr> <td>district_domain.json</td> <td></td> </tr> <tr> <td>loan_domain.json</td> <td></td> </tr> <tr> <td>order_domain.json</td> <td></td> </tr> <tr> <td>trans_domain.json</td> <td></td> </tr> <!-- Challenge data --> <tr> <td>challenge_with_id.csv</td> <td></td> </tr> <!-- Model checkpoints and other artifacts --> <tr> <td>account_label_encoders.pkl</td> <td></td> </tr> <tr> <td>card_label_encoders.pkl</td> <td></td> </tr> <tr> <td>client_label_encoders.pkl</td> <td></td> </tr> <tr> <td>disp_label_encoders.pkl</td> <td></td> </tr> <tr> <td>district_label_encoders.pkl</td> <td></td> </tr> <tr> <td>loan_label_encoders.pkl</td> <td></td> </tr> <tr> <td>order_label_encoders.pkl</td> <td></td> </tr> <tr> <td>trans_label_encoders.pkl</td> <td></td> </tr> <tr> <td>workspace/train_1/cluster_ckpt.pkl</td> <td></td> </tr> <tr> <td>workspace/train_1/models/*</td> <td></td> </tr> <!-- Synthetic data --> <tr> <td>workspace/train_1/account/_final/account_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/card/_final/card_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/client/_final/client_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/disp/_final/disp_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/district/_final/district_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/loan/_final/loan_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/order/_final/order_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/trans/_final/trans_synthetic.csv</td> <td></td> </tr> <!-- White-Box Models - Eval --> <tr> <td rowspan="27"><strong>White-Box Models - Eval</strong></td> <!-- Data domain files --> <td>account_domain.json</td> <td></td> </tr> <tr> <td>card_domain.json</td> <td></td> </tr> <tr> <td>client_domain.json</td> <td></td> </tr> <tr> <td>disp_domain.json</td> <td></td> </tr> <tr> <td>district_domain.json</td> <td></td> </tr> <tr> <td>loan_domain.json</td> <td></td> </tr> <tr> <td>order_domain.json</td> <td></td> </tr> <tr> <td>trans_domain.json</td> <td></td> </tr> <!-- Challenge data --> <tr> <td>challenge_with_id.csv</td> <td></td> </tr> <!-- Model checkpoints and other artifacts --> <tr> <td>account_label_encoders.pkl</td> <td></td> </tr> <tr> <td>card_label_encoders.pkl</td> <td></td> </tr> <tr> <td>client_label_encoders.pkl</td> <td></td> </tr> <tr> <td>disp_label_encoders.pkl</td> <td></td> </tr> <tr> <td>district_label_encoders.pkl</td> <td></td> </tr> <tr> <td>loan_label_encoders.pkl</td> <td></td> </tr> <tr> <td>order_label_encoders.pkl</td> <td></td> </tr> <tr> <td>trans_label_encoders.pkl</td> <td></td> </tr> <tr> <td>workspace/train_1/cluster_ckpt.pkl</td> <td></td> </tr> <tr> <td>workspace/train_1/models/*</td> <td></td> </tr> <!-- Synthetic data --> <tr> <td>workspace/train_1/account/_final/account_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/card/_final/card_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/client/_final/client_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/disp/_final/disp_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/district/_final/district_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/loan/_final/loan_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/order/_final/order_synthetic.csv</td> <td></td> </tr> <tr> <td>workspace/train_1/trans/_final/trans_synthetic.csv</td> <td></td> </tr> </table>

## Task

Your task as a competitor is to produce, for each model in `dev` and `final`, a CSV file listing your confidence scores (values between 0 and 1) for the membership of the challenge examples. You must save these scores in a `prediction.csv` file and place it in the same folder as the corresponding model. A submission to the challenge is an an archive containing just these `prediction.csv` files.

**You must submit predictions for both `dev` and `final` when you submit to CodaBench.**

In the following, we will show you how to compute predictions from a basic membership inference attack and package them as a submission archive. To start, let's create a baseline attack model `clavaddpm_attack_model` based on it's respective shadow models: 

In [9]:
def get_attack_model(base_train_path: Path) -> Callable[[Any], float]:
    return lambda x : random.uniform(0, 1)

base_clavaddpm_train_path = os.path.join(CLAVADDPM_DATA_DIR, "train")
clavaddpm_attack_model = get_attack_model(base_clavaddpm_train_path)

Using the attack model, we can obtain predictions for each point in the challenge point set for train, dev and eval:

In [10]:
phases = ["train", "dev", "eval"]

for base_dir, attack_model in zip([CLAVADDPM_DATA_DIR], [clavaddpm_attack_model]):
    for phase in phases:
        root = os.path.join(base_dir, phase)
        for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
            path = os.path.join(root, model_folder)
    
            challenge_points = get_challenge_points(path)
    
            predictions = torch.Tensor([attack_model(cp) for cp in challenge_points])
           
            assert torch.all((0 <= predictions) & (predictions <= 1))
            with open(os.path.join(path, "prediction.csv"), mode="w", newline="") as file:
                writer = csv.writer(file)
    
                # Write each value in a separate row
                for value in list(predictions.numpy().squeeze()):
                    writer.writerow([value])

## Scoring

Let's see how the attack does on `train`, for which we have the ground truth.
When preparing a submission, you can use part of `train` to develop an attack and a held-out part to evaluate your attack.

In [11]:
tpr_at_fpr_list = []
for base_dir in [CLAVADDPM_DATA_DIR]:
    predictions = []
    solutions  = []
    root = os.path.join(base_dir, "train")
    for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
        path = os.path.join(root, model_folder)
        predictions.append(np.loadtxt(os.path.join(path, "prediction.csv")))
        solutions.append(np.loadtxt(os.path.join(path, "challenge_label.csv"), skiprows=1))
    
    predictions = np.concatenate(predictions)
    solutions = np.concatenate(solutions)
    
    tpr_at_fpr = get_tpr_at_fpr(solutions, predictions)
    tpr_at_fpr_list.append(tpr_at_fpr)
    
    print(f"{base_dir.split("_")[0]} Train Attack TPR at FPR==10%: {tpr_at_fpr}")

final_tpr_at_fpr = max(tpr_at_fpr_list)
print(f"Final Train Attack TPR at FPR==10%: {final_tpr_at_fpr}")

clavaddpm Train Attack TPR at FPR==10%: 0.105
Final Train Attack TPR at FPR==10%: 0.105


## Packaging the submission

Now we can store the predictions into a zip file, which you can submit to CodaBench. We create seperate zip files for dev and eval. The structure of the submission is as follows:
```
└── root_folder
    ├── clavaddpm_white_box
       ├── dev
       │   └── clavaddpm_#
       │       └── prediction.csv
       └── eval
           └── clavaddpm_#
                └── prediction.csv
```

In [12]:
for phase in ["dev", "eval"]:
    with zipfile.ZipFile(f"white_box_{phase}_multi_table_submission.zip", 'w') as zipf:
        for base_dir in [CLAVADDPM_DATA_DIR]:
            root = os.path.join(base_dir, phase)
            for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
                path = os.path.join(root, model_folder)
                if not os.path.isdir(path): continue

                file = os.path.join(path, "prediction.csv")
                if os.path.exists(file):
                    # Use `arcname` to remove the base directory and phase directory from the zip path
                    arcname = os.path.relpath(file, os.path.dirname(base_dir))
                    zipf.write(file, arcname=arcname)
                else:
                    raise FileNotFoundError(f"`prediction.csv` not found in {path}.")

The generated white_box_dev_multi_table_submission.zip and white_box_eval_multi_table_submission.zip can be used to directly submit to the respective phases in the CodaBench UI.